In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import cv2
import pytesseract
from PIL import Image
import os
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt


In [6]:
import glob
import pandas as pd
import os

folder_path = "/content/drive/MyDrive/Colab Notebooks/SIA/HRC_WHU"

In [5]:
# Load TIFF image
img_path = 'image.tif'
img = cv2.imread(img_path)

# Check if image is loaded correctly
if img is None:
    print(f"Error: Unable to load image '{img_path}'")
else:
    # Preprocess image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply OCR
    text = pytesseract.image_to_string(gray)

    # Save text to file
    with open('output.txt', 'w') as f:
        f.write(text)

Error: Unable to load image 'image.tif'


In [7]:
# Folder path containing TIFF files
folder_path = '/content/drive/MyDrive/Colab Notebooks/SIA/HRC_WHU'

# Loop through TIFF files and convert to JPG
for filename in os.listdir(folder_path):
    if filename.endswith(".tif"):
        img = cv2.imread(os.path.join(folder_path, filename))
        cv2.imwrite(os.path.join(folder_path, filename.replace(".tif", ".jpg")), img)


In [8]:
import glob
image_paths = glob.glob(folder_path + '/*.jpg')
df = pd.DataFrame({'image_path': image_paths})

In [9]:
# Convert images to text using OCR
def image_to_text(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text

In [10]:
# Apply OCR to images
df['text'] = df['image_path'].apply(image_to_text)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Assuming labels are based on the filename 'label_image.jpg', need to extract them.
def extract_label(image_path):
    filename = os.path.basename(image_path)  # Get filename from path
    label = filename.split('_')[0]  # Split filename and take the first part
    return label

# Apply the function to create the 'label' column
df['label'] = df['image_path'].apply(extract_label)

# NLP preprocessing
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['text'])
y = df['label']

In [12]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Train NLP model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spac

In [14]:
# Split data into training and testing sets

X_train_text, X_test_text, y_train, y_test = train_test_split(df['text'], y, test_size=0.2, random_state=42) # Split the original text data

# Train NLP model
# Now encode the text data
train_encodings = tokenizer.batch_encode_plus(X_train_text.tolist(), add_special_tokens=True, max_length=512, return_attention_mask=True, return_tensors='pt', padding='max_length', truncation=True) # Use tolist() to convert to a list
test_encodings = tokenizer.batch_encode_plus(X_test_text.tolist(), add_special_tokens=True, max_length=512, return_attention_mask=True, return_tensors='pt', padding='max_length', truncation=True) # Use tolist() to convert to a list

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
import torch
from sklearn.preprocessing import LabelEncoder
# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the encoder to your training labels and transform them
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)  # Transform test labels using the same encoder

# Train model or use y_train_encoded and y_test_encoded in the training loop:
for epoch in range(5):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_encodings['input_ids'].to(device), attention_mask=train_encodings['attention_mask'].to(device), labels=torch.tensor(y_train_encoded).to(device)) # Use encoded labels
    loss = criterion(outputs.logits, torch.tensor(y_train_encoded).to(device)) # Use encoded labels
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [ ]:
# Evaluate model
model.eval()
with torch.no_grad():
    outputs = model(test_encodings['input_ids'].to(device), attention_mask=test_encodings['attention_mask'].to(device))
    logits = outputs.logits
    predicted = torch.argmax(logits, dim=1)
    accuracy = accuracy_score(y_test, predicted.cpu().numpy())
    print(f'Test Accuracy: {accuracy:.4f}')
    print(classification_report(y_test, predicted.cpu().numpy()))

In [ ]:
# Visualization
plt.bar(df['label'].value_counts().index, df['label'].value_counts())
plt.xlabel('Label')
plt.ylabel('Count')
plt.title('Label Distribution')
plt.show()